In [ ]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk

import sys

sys.path.insert(0, '../../dataloaders/')
import niftiio as nio

In [ ]:
TR_IMAGE='./imageTr/'
TR_LABEL='./imageTr/'
TASK_JSON='task1_dataset.json'

OUT_FOLDER='./tmp_normalized/'

TASK='./task1_dataset.json'

In [ ]:
import json

with open(TASK, 'r') as f:
    data = json.load(f)

imgs, segs = zip(*[ (d['image'], d['label']) for d in data['training']])

# imgs = sorted(glob.glob(TR_IMAGE + '/*.nii.gz'))
# segs = sorted(glob.glob(TR_LABEL + '/*.nii.gz'))

pids = [ name.split('.')[0][5:] for name in imgs]

In [ ]:
# helper function
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [ ]:
import copy
scan_dir = OUT_FOLDER
# window and width -> low and high
LIR = -75
HIR = 215
os.makedirs(scan_dir, exist_ok = True)

reindex = 0
for img_fid, seg_fid, pid in zip(imgs, segs, pids):

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    array = sitk.GetArrayFromImage(img_obj)

    array[array > HIR] = HIR
    array[array < LIR] = LIR
    
    array = (array - array.min()) / (array.max() - array.min()) * 255.0
    
    # then normalize this
    
    wined_img = sitk.GetImageFromArray(array)
    wined_img = copy_spacing_ori(img_obj, wined_img)
    
    out_img_fid = os.path.join( scan_dir, f'image_{str(reindex)}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{str(reindex)}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(wined_img, out_img_fid, True) 
    sitk.WriteImage(seg_obj, out_lb_fid, True) 
    print('From', img_fid, seg_fid, 'Saved', out_img_fid, out_lb_fid)
    reindex += 1
